## goal of this note book is to scale down the food101 dataset to 3 classees and Reduced size 


In [26]:
#importing the necessary Librarise
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [27]:
#loading the data
train = datasets.Food101(
    root = './data',
    split = 'train',
    download = True)
test = datasets.Food101(
    root = './data',
    split = 'test',
    download = True)

100%|██████████| 5.00G/5.00G [03:26<00:00, 24.2MB/s] 


### we got the data that we need thats a 5GB data
* for simplicity sake we are going to use 3 classes and 10% of the random images


In [28]:
from pathlib import Path

In [29]:
import random
import pathlib

data_path = pathlib.Path('./data/food-101/images/')
data_dir = pathlib.Path('./data/food-101/')
target_classes = ['pizza', 'sushi', 'steak']
amount_to_get = 0.2

def get_subset(image_path=data_path,
               data_splits=["train", "test"], 
               target_classes=["pizza", "steak", "sushi"],
               amount=0.1,
               seed=42):
    random.seed(seed)
    label_splits = {}
    
    for data_split in data_splits:
        print(f"[INFO] Creating image split for: {data_split}...")

        label_path = data_dir / "meta" / f"{data_split}.txt"
        with open(label_path, "r") as f:
            labels = [line.strip() for line in f if line.split("/")[0] in target_classes]
        
        number_to_sample = round(amount * len(labels))
        print(f"[INFO] Getting random subset of {number_to_sample} images for {data_split}...")
        sampled_images = random.sample(labels, k=number_to_sample)
        
        image_paths = [image_path / f"{sample_image}.jpg" for sample_image in sampled_images]
        label_splits[data_split] = image_paths

    return label_splits

label_splits = get_subset(amount=amount_to_get)
print(label_splits["train"][:10])


[INFO] Creating image split for: train...
[INFO] Getting random subset of 450 images for train...
[INFO] Creating image split for: test...
[INFO] Getting random subset of 150 images for test...
[PosixPath('data/food-101/images/pizza/3269634.jpg'), PosixPath('data/food-101/images/pizza/1524655.jpg'), PosixPath('data/food-101/images/steak/2825100.jpg'), PosixPath('data/food-101/images/steak/225990.jpg'), PosixPath('data/food-101/images/steak/1839481.jpg'), PosixPath('data/food-101/images/pizza/38349.jpg'), PosixPath('data/food-101/images/pizza/3018077.jpg'), PosixPath('data/food-101/images/sushi/93139.jpg'), PosixPath('data/food-101/images/pizza/2702825.jpg'), PosixPath('data/food-101/images/sushi/200025.jpg')]


In [30]:
# Create target directory path
target_dir_name = f"../data/pizza_steak_sushi_{str(int(amount_to_get*100))}_percent"
print(f"Creating directory: '{target_dir_name}'")

# Setup the directories
target_dir = pathlib.Path(target_dir_name)

# Make the directories
target_dir.mkdir(parents=True, exist_ok=True)

Creating directory: '../data/pizza_steak_sushi_20_percent'


In [31]:
import shutil

for image_split in label_splits.keys():
    for image_path in label_splits[str(image_split)]:
        dest_dir = target_dir / image_split / image_path.parent.stem / image_path.name
        if not dest_dir.parent.is_dir():
            dest_dir.parent.mkdir(parents=True, exist_ok=True)
        print(f"[INFO] Copying {image_path} to {dest_dir}...")
        shutil.copy2(image_path, dest_dir)

[INFO] Copying data/food-101/images/pizza/3269634.jpg to ../data/pizza_steak_sushi_20_percent/train/pizza/3269634.jpg...
[INFO] Copying data/food-101/images/pizza/1524655.jpg to ../data/pizza_steak_sushi_20_percent/train/pizza/1524655.jpg...
[INFO] Copying data/food-101/images/steak/2825100.jpg to ../data/pizza_steak_sushi_20_percent/train/steak/2825100.jpg...
[INFO] Copying data/food-101/images/steak/225990.jpg to ../data/pizza_steak_sushi_20_percent/train/steak/225990.jpg...
[INFO] Copying data/food-101/images/steak/1839481.jpg to ../data/pizza_steak_sushi_20_percent/train/steak/1839481.jpg...
[INFO] Copying data/food-101/images/pizza/38349.jpg to ../data/pizza_steak_sushi_20_percent/train/pizza/38349.jpg...
[INFO] Copying data/food-101/images/pizza/3018077.jpg to ../data/pizza_steak_sushi_20_percent/train/pizza/3018077.jpg...
[INFO] Copying data/food-101/images/sushi/93139.jpg to ../data/pizza_steak_sushi_20_percent/train/sushi/93139.jpg...
[INFO] Copying data/food-101/images/pizza/

In [32]:
# Check lengths of directories
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  import os
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
    
walk_through_dir(target_dir)

There are 2 directories and 0 images in '../data/pizza_steak_sushi_20_percent'.
There are 3 directories and 0 images in '../data/pizza_steak_sushi_20_percent/test'.
There are 0 directories and 58 images in '../data/pizza_steak_sushi_20_percent/test/steak'.
There are 0 directories and 46 images in '../data/pizza_steak_sushi_20_percent/test/sushi'.
There are 0 directories and 46 images in '../data/pizza_steak_sushi_20_percent/test/pizza'.
There are 3 directories and 0 images in '../data/pizza_steak_sushi_20_percent/train'.
There are 0 directories and 146 images in '../data/pizza_steak_sushi_20_percent/train/steak'.
There are 0 directories and 150 images in '../data/pizza_steak_sushi_20_percent/train/sushi'.
There are 0 directories and 154 images in '../data/pizza_steak_sushi_20_percent/train/pizza'.


In [33]:
# Zip pizza_steak_sushi images
zip_file_name = data_dir / f"pizza_steak_sushi_{str(int(amount_to_get*100))}_percent"
shutil.make_archive(zip_file_name, 
                    format="zip", 
                    root_dir=target_dir)

'/Users/bhushanchowdary/Documents/GitHub/PyTorch_fundamentals/pytoch_coustomdatasets/data/food-101/pizza_steak_sushi_20_percent.zip'

In [34]:
!ls -la ../data/


total 0
drwxr-xr-x@  4 bhushanchowdary  staff  128 Jun 24 13:29 .
drwxr-xr-x@ 24 bhushanchowdary  staff  768 Jun 24 13:30 ..
drwxr-xr-x@  3 bhushanchowdary  staff   96 Jun 23 11:57 FashionMNIST
drwxr-xr-x@  4 bhushanchowdary  staff  128 Jun 24 13:30 pizza_steak_sushi_20_percent


In [38]:
!mkdir -p pizza_steak_sushi
!unzip /Users/bhushanchowdary/Documents/GitHub/PyTorch_fundamentals/pytoch_coustomdatasets/data/food-101/pizza_steak_sushi_20_percent.zip -d pizza_steak_sushi


Archive:  /Users/bhushanchowdary/Documents/GitHub/PyTorch_fundamentals/pytoch_coustomdatasets/data/food-101/pizza_steak_sushi_20_percent.zip
   creating: /Users/bhushanchowdary/Documents/GitHub/PyTorch_fundamentals/pytoch_coustomdatasets/pizza_steak_sushi/test
   creating: /Users/bhushanchowdary/Documents/GitHub/PyTorch_fundamentals/pytoch_coustomdatasets/pizza_steak_sushi/train
   creating: /Users/bhushanchowdary/Documents/GitHub/PyTorch_fundamentals/pytoch_coustomdatasets/pizza_steak_sushi/test/pizza
   creating: /Users/bhushanchowdary/Documents/GitHub/PyTorch_fundamentals/pytoch_coustomdatasets/pizza_steak_sushi/test/steak
   creating: /Users/bhushanchowdary/Documents/GitHub/PyTorch_fundamentals/pytoch_coustomdatasets/pizza_steak_sushi/test/sushi
  inflating: pizza_steak_sushi/test/steak/413497.jpg  
  inflating: pizza_steak_sushi/test/steak/2246332.jpg  
  inflating: pizza_steak_sushi/test/steak/2475366.jpg  
  inflating: pizza_steak_sushi/test/steak/1982192.jpg  
  inflating: pizz